# ETAPA 6: Análise Qualitativa das Respostas Abertas

## Consulta à Comunidade UFPE sobre o Uso de Inteligência Artificial

---

**Objetivo:** Explorar percepções, preocupações e sugestões em profundidade.

**Variáveis analisadas (7 questões abertas):**
1. Benefícios percebidos no uso da IA
2. Riscos e preocupações identificados
3. Impacto na produção autoral
4. Impacto na integridade acadêmica
5. Medidas para uso ético
6. Estrutura de governança
7. Sugestões e comentários finais

**Procedimentos:**
1. Realizar leitura exploratória de amostra representativa (mínimo 10% das respostas)
2. Desenvolver codebook inicial com categorias temáticas emergentes
3. Aplicar Análise de Conteúdo (Bardin) para categorização sistemática
4. Utilizar técnicas de análise textual para identificação de temas
5. Gerar nuvens de palavras e análises de frequência
6. Selecionar citações representativas para cada tema identificado
7. Estratificar análise por vínculo institucional quando pertinente

**Produto:** Relatório de análise de conteúdo com categorias, frequências, citações exemplares e visualizações.

**Critério de Validação:** Saturação temática; coerência entre categorias e citações.

---

## Requisitos Técnicos

### Kernel Python
- **Python:** 3.9 ou superior
- **Kernel recomendado:** `python3` ou ambiente virtual com as dependências instaladas

### Dados de Entrada

| Arquivo | Descrição | Origem |
|---------|-----------|--------|
| `dados_limpos_etapa1.xlsx` | Base de dados limpa com 2.164 registros | Etapa 1 |

### Dados de Saída

| Arquivo | Descrição |
|---------|----------|
| `qualitativa_etapa6.xlsx` | Análise de frequência de termos e categorias temáticas |
| `amostra_analise_etapa6.xlsx` | Amostra de 10% para análise manual |
| `grafico_nuvem_[variavel]_etapa6.png` | Nuvens de palavras por questão |
| `grafico_frequencia_termos_etapa6.png` | Termos mais frequentes por questão |
| `grafico_comprimento_respostas_etapa6.png` | Distribuição do comprimento das respostas |
| `citacoes_representativas_etapa6.xlsx` | Citações selecionadas por tema |
| `relatorio_qualitativa_etapa6.md` | Relatório com análise de conteúdo |

---

## 1. Instalação de Dependências

In [ ]:
# ============================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# Execute esta célula apenas uma vez para instalar os pacotes necessários
# ============================================================================

!pip install pandas openpyxl matplotlib seaborn numpy wordcloud nltk unidecode --quiet

print("✅ Dependências instaladas com sucesso!")

## 2. Configuração Inicial

In [ ]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
import re
import string
import warnings
warnings.filterwarnings('ignore')

# Tentar importar bibliotecas opcionais
try:
    from wordcloud import WordCloud
    WORDCLOUD_DISPONIVEL = True
except ImportError:
    WORDCLOUD_DISPONIVEL = False
    print("⚠️ Biblioteca 'wordcloud' não disponível.")

try:
    from unidecode import unidecode
    UNIDECODE_DISPONIVEL = True
except ImportError:
    UNIDECODE_DISPONIVEL = False
    print("⚠️ Biblioteca 'unidecode' não disponível.")

try:
    import nltk
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    NLTK_DISPONIVEL = True
except:
    NLTK_DISPONIVEL = False
    print("⚠️ Biblioteca 'nltk' não disponível ou erro no download.")

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
sns.set_style("whitegrid")

print("\nBibliotecas carregadas com sucesso!")
print(f"  • Pandas: {pd.__version__}")
print(f"  • WordCloud: {'Disponível' if WORDCLOUD_DISPONIVEL else 'Não disponível'}")
print(f"  • NLTK: {'Disponível' if NLTK_DISPONIVEL else 'Não disponível'}")
print(f"  • Unidecode: {'Disponível' if UNIDECODE_DISPONIVEL else 'Não disponível'}")

In [ ]:
# ============================================================================
# CONFIGURAÇÃO DE ARQUIVOS DE ENTRADA E SAÍDA
# ============================================================================

# Arquivo de entrada (gerado na Etapa 1)
ARQUIVO_ENTRADA = "dados_limpos_etapa1.xlsx"

# Arquivos de saída
ARQUIVO_SAIDA_EXCEL = "qualitativa_etapa6.xlsx"
ARQUIVO_SAIDA_AMOSTRA = "amostra_analise_etapa6.xlsx"
ARQUIVO_SAIDA_CITACOES = "citacoes_representativas_etapa6.xlsx"
ARQUIVO_SAIDA_RELATORIO = "relatorio_qualitativa_etapa6.md"

# Percentual da amostra para análise manual
PERCENTUAL_AMOSTRA = 0.10  # 10%

# Seed para reprodutibilidade
SEED = 42

print("Configuração de arquivos:")
print(f"  📥 Entrada: {ARQUIVO_ENTRADA}")
print(f"  📤 Saídas:  {ARQUIVO_SAIDA_EXCEL}")
print(f"            {ARQUIVO_SAIDA_AMOSTRA}")
print(f"            {ARQUIVO_SAIDA_CITACOES}")
print(f"            {ARQUIVO_SAIDA_RELATORIO}")
print(f"  📊 Amostra para análise manual: {PERCENTUAL_AMOSTRA*100:.0f}%")

## 3. Carregamento dos Dados

In [ ]:
# Carregar dados limpos da Etapa 1
try:
    df = pd.read_excel(ARQUIVO_ENTRADA)
    print(f"✅ Dados carregados com sucesso!")
    print(f"   Registros: {len(df)}")
    print(f"   Variáveis: {len(df.columns)}")
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{ARQUIVO_ENTRADA}' não encontrado!")
    print(f"   Certifique-se de que a Etapa 1 foi executada e o arquivo está no diretório correto.")
    raise

In [ ]:
# ============================================================================
# IDENTIFICAÇÃO DAS VARIÁVEIS ABERTAS
# ============================================================================

# Dicionário com as variáveis abertas e suas descrições
variaveis_abertas = {
    'Beneficios_Percebidos': {
        'descricao': 'Quais benefícios você percebe no uso de IA?',
        'nome_curto': 'Benefícios'
    },
    'Riscos_Preocupacoes': {
        'descricao': 'Quais riscos e preocupações você identifica?',
        'nome_curto': 'Riscos'
    },
    'Impacto_Producao_Autoral': {
        'descricao': 'Como a IA impacta a produção autoral?',
        'nome_curto': 'Produção Autoral'
    },
    'Impacto_Integridade_Academica': {
        'descricao': 'Como a IA impacta a integridade acadêmica?',
        'nome_curto': 'Integridade Acadêmica'
    },
    'Medidas_Uso_Etico': {
        'descricao': 'Quais medidas devem ser adotadas para uso ético?',
        'nome_curto': 'Medidas Éticas'
    },
    'Estrutura_Governanca': {
        'descricao': 'Como deve ser a estrutura de governança?',
        'nome_curto': 'Governança'
    },
    'Comentarios_Finais': {
        'descricao': 'Sugestões e comentários finais',
        'nome_curto': 'Comentários Finais'
    }
}

print("=" * 80)
print("VARIÁVEIS ABERTAS PARA ANÁLISE QUALITATIVA")
print("=" * 80)

estatisticas_variaveis = []

for var, info in variaveis_abertas.items():
    n_respostas = df[var].notna().sum()
    pct_respostas = n_respostas / len(df) * 100
    
    # Calcular comprimento médio das respostas
    comprimentos = df[var].dropna().astype(str).str.len()
    comp_medio = comprimentos.mean() if len(comprimentos) > 0 else 0
    
    estatisticas_variaveis.append({
        'Variável': var,
        'Nome Curto': info['nome_curto'],
        'Respostas': n_respostas,
        '% Preenchimento': round(pct_respostas, 1),
        'Comp. Médio (chars)': round(comp_medio, 0)
    })
    
    print(f"\n📌 {info['nome_curto']}")
    print(f"   Variável: {var}")
    print(f"   Respostas válidas: {n_respostas} ({pct_respostas:.1f}%)")
    print(f"   Comprimento médio: {comp_medio:.0f} caracteres")

df_estatisticas = pd.DataFrame(estatisticas_variaveis)

---

## 4. Funções Auxiliares para Análise Textual

In [ ]:
# ============================================================================
# STOPWORDS EM PORTUGUÊS (lista expandida)
# ============================================================================

# Stopwords básicas em português
STOPWORDS_PT = {
    'a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo',
    'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas',
    'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele',
    'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse',
    'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava',
    'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes',
    'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram',
    'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem',
    'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram',
    'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui',
    'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei',
    'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram',
    'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos',
    'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse',
    'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'lhe', 'lhes', 'lo',
    'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito',
    'muita', 'muitas', 'muitos', 'na', 'não', 'nas', 'nem', 'no', 'nos',
    'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'o', 'os',
    'ou', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando',
    'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'ser',
    'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos',
    'seu', 'seus', 'só', 'somos', 'sou', 'sua', 'suas', 'também', 'te',
    'tem', 'tém', 'temos', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho',
    'ter', 'terá', 'terão', 'terei', 'teremos', 'teria', 'teriam',
    'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tínhamos',
    'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem',
    'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tu', 'tua', 'tuas',
    'um', 'uma', 'umas', 'uns', 'você', 'vocês', 'vos',
    # Adicionais comuns em respostas de pesquisa
    'ser', 'pode', 'podem', 'poder', 'fazer', 'uso', 'usar', 'utilizar',
    'forma', 'modo', 'tipo', 'coisa', 'coisas', 'vez', 'vezes', 'caso',
    'casos', 'exemplo', 'alguns', 'algumas', 'algo', 'ainda', 'assim',
    'bem', 'cada', 'onde', 'porque', 'pois', 'então', 'porém', 'contudo',
    'apenas', 'sobre', 'todo', 'toda', 'todos', 'todas', 'outro', 'outra',
    'outros', 'outras', 'mesmo', 'mesma', 'mesmos', 'mesmas', 'próprio',
    'própria', 'dia', 'dias', 'ano', 'anos', 'parte', 'partes', 'lado',
    'grande', 'maior', 'menor', 'melhor', 'pior', 'bom', 'boa', 'bons',
    'boas', 'mau', 'má', 'maus', 'más', 'novo', 'nova', 'novos', 'novas',
    'primeiro', 'primeira', 'segundo', 'segunda', 'último', 'última',
    'sim', 'nao', 'talvez', 'sempre', 'nunca', 'aqui', 'ali', 'lá', 'cá',
    'agora', 'hoje', 'ontem', 'amanhã', 'antes', 'depois', 'durante',
    'while', 'the', 'and', 'of', 'to', 'in', 'is', 'it', 'that', 'for',
    'acredito', 'acho', 'penso', 'creio', 'entendo', 'considero',
    'ia', 'inteligência', 'artificial', 'inteligencia', 'ufpe'
}

# Adicionar stopwords do NLTK se disponível
if NLTK_DISPONIVEL:
    try:
        STOPWORDS_PT = STOPWORDS_PT.union(set(stopwords.words('portuguese')))
    except:
        pass

print(f"✅ {len(STOPWORDS_PT)} stopwords carregadas")

In [ ]:
# ============================================================================
# FUNÇÕES DE PRÉ-PROCESSAMENTO DE TEXTO
# ============================================================================

def limpar_texto(texto):
    """
    Limpa e normaliza um texto para análise.
    """
    if pd.isna(texto):
        return ""
    
    texto = str(texto).lower()
    
    # Remover acentos se unidecode disponível
    if UNIDECODE_DISPONIVEL:
        texto = unidecode(texto)
    
    # Remover pontuação e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', texto)
    
    # Remover espaços múltiplos
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto


def tokenizar(texto, remover_stopwords=True, min_length=3):
    """
    Tokeniza um texto em palavras individuais.
    """
    texto_limpo = limpar_texto(texto)
    
    if not texto_limpo:
        return []
    
    # Tokenizar
    palavras = texto_limpo.split()
    
    # Filtrar
    if remover_stopwords:
        palavras = [p for p in palavras if p not in STOPWORDS_PT]
    
    # Filtrar por comprimento mínimo
    palavras = [p for p in palavras if len(p) >= min_length]
    
    return palavras


def extrair_ngramas(texto, n=2):
    """
    Extrai n-gramas de um texto.
    """
    palavras = tokenizar(texto, remover_stopwords=False, min_length=2)
    
    if len(palavras) < n:
        return []
    
    ngramas = []
    for i in range(len(palavras) - n + 1):
        ngrama = ' '.join(palavras[i:i+n])
        # Verificar se pelo menos uma palavra não é stopword
        palavras_ngrama = ngrama.split()
        if any(p not in STOPWORDS_PT for p in palavras_ngrama):
            ngramas.append(ngrama)
    
    return ngramas


def calcular_frequencia_termos(serie, top_n=30, min_freq=5):
    """
    Calcula a frequência de termos em uma série de textos.
    """
    todas_palavras = []
    
    for texto in serie.dropna():
        palavras = tokenizar(texto)
        todas_palavras.extend(palavras)
    
    contagem = Counter(todas_palavras)
    
    # Filtrar por frequência mínima
    contagem = {k: v for k, v in contagem.items() if v >= min_freq}
    
    # Criar DataFrame
    df_freq = pd.DataFrame(contagem.items(), columns=['Termo', 'Frequência'])
    df_freq = df_freq.sort_values('Frequência', ascending=False).head(top_n)
    df_freq['Ranking'] = range(1, len(df_freq) + 1)
    df_freq = df_freq[['Ranking', 'Termo', 'Frequência']]
    
    return df_freq


def calcular_frequencia_bigramas(serie, top_n=20, min_freq=5):
    """
    Calcula a frequência de bigramas em uma série de textos.
    """
    todos_bigramas = []
    
    for texto in serie.dropna():
        bigramas = extrair_ngramas(texto, n=2)
        todos_bigramas.extend(bigramas)
    
    contagem = Counter(todos_bigramas)
    
    # Filtrar por frequência mínima
    contagem = {k: v for k, v in contagem.items() if v >= min_freq}
    
    # Criar DataFrame
    df_freq = pd.DataFrame(contagem.items(), columns=['Bigrama', 'Frequência'])
    df_freq = df_freq.sort_values('Frequência', ascending=False).head(top_n)
    df_freq['Ranking'] = range(1, len(df_freq) + 1)
    df_freq = df_freq[['Ranking', 'Bigrama', 'Frequência']]
    
    return df_freq


print("✅ Funções de pré-processamento carregadas com sucesso!")

In [ ]:
# ============================================================================
# FUNÇÕES PARA SELEÇÃO DE CITAÇÕES REPRESENTATIVAS
# ============================================================================

def selecionar_citacoes(serie, termos_busca=None, n_citacoes=5, min_chars=50, max_chars=500):
    """
    Seleciona citações representativas de uma série de textos.
    Se termos_busca fornecidos, prioriza respostas que contenham esses termos.
    """
    respostas = serie.dropna().astype(str)
    
    # Filtrar por comprimento
    respostas = respostas[(respostas.str.len() >= min_chars) & (respostas.str.len() <= max_chars)]
    
    if len(respostas) == 0:
        return []
    
    if termos_busca:
        # Priorizar respostas com termos de busca
        def contem_termos(texto):
            texto_lower = texto.lower()
            return sum(1 for termo in termos_busca if termo.lower() in texto_lower)
        
        respostas_ordenadas = respostas.copy()
        scores = respostas_ordenadas.apply(contem_termos)
        respostas_ordenadas = respostas_ordenadas[scores > 0]
        
        if len(respostas_ordenadas) >= n_citacoes:
            return respostas_ordenadas.sample(n=n_citacoes, random_state=SEED).tolist()
    
    # Selecionar aleatoriamente
    n_selecionar = min(n_citacoes, len(respostas))
    return respostas.sample(n=n_selecionar, random_state=SEED).tolist()


def selecionar_citacoes_por_vinculo(df, variavel, n_por_vinculo=2):
    """
    Seleciona citações representativas estratificadas por vínculo.
    """
    citacoes = []
    
    for vinculo in df['Vinculo_Padronizado'].dropna().unique():
        df_vinculo = df[df['Vinculo_Padronizado'] == vinculo]
        cits = selecionar_citacoes(df_vinculo[variavel], n_citacoes=n_por_vinculo)
        
        for cit in cits:
            citacoes.append({
                'Vínculo': vinculo,
                'Citação': cit
            })
    
    return pd.DataFrame(citacoes)


print("✅ Funções de seleção de citações carregadas com sucesso!")

---

## 5. Análise Exploratória das Respostas

In [ ]:
# ============================================================================
# ESTATÍSTICAS DESCRITIVAS DAS RESPOSTAS
# ============================================================================

print("=" * 80)
print("ESTATÍSTICAS DESCRITIVAS DAS RESPOSTAS ABERTAS")
print("=" * 80)

print(df_estatisticas.to_string(index=False))

In [ ]:
# ============================================================================
# DISTRIBUIÇÃO DO COMPRIMENTO DAS RESPOSTAS
# ============================================================================

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, (var, info) in enumerate(variaveis_abertas.items()):
    ax = axes[idx]
    
    comprimentos = df[var].dropna().astype(str).str.len()
    
    if len(comprimentos) > 0:
        ax.hist(comprimentos, bins=30, color='steelblue', edgecolor='white', alpha=0.7)
        ax.axvline(comprimentos.median(), color='red', linestyle='--', label=f'Mediana: {comprimentos.median():.0f}')
        ax.set_title(info['nome_curto'], fontsize=10, fontweight='bold')
        ax.set_xlabel('Caracteres')
        ax.set_ylabel('Frequência')
        ax.legend(fontsize=8)
    else:
        ax.text(0.5, 0.5, 'Sem dados', ha='center', va='center')
        ax.set_title(info['nome_curto'], fontsize=10)

# Remover subplot extra
axes[7].axis('off')

plt.suptitle('Distribuição do Comprimento das Respostas (em caracteres)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('grafico_comprimento_respostas_etapa6.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_comprimento_respostas_etapa6.png")

---

## 6. Análise de Frequência de Termos

In [ ]:
# ============================================================================
# FREQUÊNCIA DE TERMOS PARA CADA QUESTÃO
# ============================================================================

print("=" * 80)
print("TERMOS MAIS FREQUENTES POR QUESTÃO")
print("=" * 80)

frequencias_termos = {}

for var, info in variaveis_abertas.items():
    print(f"\n📌 {info['nome_curto']}")
    print("-" * 60)
    
    freq = calcular_frequencia_termos(df[var], top_n=15, min_freq=10)
    frequencias_termos[var] = freq
    
    if len(freq) > 0:
        print(freq.to_string(index=False))
    else:
        print("   Nenhum termo com frequência suficiente.")

In [ ]:
# ============================================================================
# GRÁFICO DE TERMOS MAIS FREQUENTES (TOP 10 DE CADA)
# ============================================================================

fig, axes = plt.subplots(2, 4, figsize=(18, 10))
axes = axes.flatten()

cores = plt.cm.Set2(np.linspace(0, 1, 7))

for idx, (var, info) in enumerate(variaveis_abertas.items()):
    ax = axes[idx]
    
    freq = frequencias_termos[var].head(10)
    
    if len(freq) > 0:
        bars = ax.barh(freq['Termo'], freq['Frequência'], color=cores[idx], edgecolor='white')
        ax.invert_yaxis()
        ax.set_xlabel('Frequência')
        ax.set_title(info['nome_curto'], fontsize=11, fontweight='bold')
        
        # Labels nas barras
        for bar in bars:
            ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
                    f'{int(bar.get_width())}', va='center', fontsize=8)
    else:
        ax.text(0.5, 0.5, 'Dados insuficientes', ha='center', va='center')
        ax.set_title(info['nome_curto'], fontsize=11)

# Remover subplot extra
axes[7].axis('off')

plt.suptitle('Top 10 Termos Mais Frequentes por Questão', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('grafico_frequencia_termos_etapa6.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_frequencia_termos_etapa6.png")

---

## 7. Nuvens de Palavras

In [ ]:
# ============================================================================
# NUVENS DE PALAVRAS PARA CADA QUESTÃO
# ============================================================================

if WORDCLOUD_DISPONIVEL:
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    
    colormaps = ['Blues', 'Greens', 'Oranges', 'Purples', 'Reds', 'YlOrBr', 'BuGn']
    
    for idx, (var, info) in enumerate(variaveis_abertas.items()):
        ax = axes[idx]
        
        # Concatenar todas as respostas
        textos = df[var].dropna().astype(str)
        texto_completo = ' '.join([limpar_texto(t) for t in textos])
        
        if texto_completo.strip():
            # Gerar nuvem de palavras
            wordcloud = WordCloud(
                width=800, height=400,
                background_color='white',
                stopwords=STOPWORDS_PT,
                colormap=colormaps[idx],
                max_words=100,
                min_font_size=10,
                random_state=SEED
            ).generate(texto_completo)
            
            ax.imshow(wordcloud, interpolation='bilinear')
            ax.axis('off')
            ax.set_title(info['nome_curto'], fontsize=12, fontweight='bold')
            
            # Salvar nuvem individual
            fig_individual, ax_individual = plt.subplots(figsize=(10, 5))
            ax_individual.imshow(wordcloud, interpolation='bilinear')
            ax_individual.axis('off')
            ax_individual.set_title(info['nome_curto'], fontsize=14, fontweight='bold')
            plt.tight_layout()
            nome_arquivo = f"grafico_nuvem_{var.lower()}_etapa6.png"
            plt.savefig(nome_arquivo, dpi=150, bbox_inches='tight')
            plt.close(fig_individual)
        else:
            ax.text(0.5, 0.5, 'Sem dados', ha='center', va='center')
            ax.set_title(info['nome_curto'], fontsize=12)
            ax.axis('off')
    
    # Remover subplot extra
    axes[7].axis('off')
    
    plt.suptitle('Nuvens de Palavras - Respostas Abertas', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('grafico_nuvens_completo_etapa6.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nNuvens de palavras salvas!")
else:
    print("⚠️ Nuvens de palavras não geradas (biblioteca 'wordcloud' não disponível).")

---

## 8. Análise de Bigramas (Expressões Frequentes)

In [ ]:
# ============================================================================
# BIGRAMAS MAIS FREQUENTES
# ============================================================================

print("=" * 80)
print("EXPRESSÕES (BIGRAMAS) MAIS FREQUENTES")
print("=" * 80)

frequencias_bigramas = {}

for var, info in variaveis_abertas.items():
    print(f"\n📌 {info['nome_curto']}")
    print("-" * 60)
    
    freq_bi = calcular_frequencia_bigramas(df[var], top_n=10, min_freq=10)
    frequencias_bigramas[var] = freq_bi
    
    if len(freq_bi) > 0:
        print(freq_bi.to_string(index=False))
    else:
        print("   Nenhum bigrama com frequência suficiente.")

---

## 9. Geração de Amostra para Análise Manual

In [ ]:
# ============================================================================
# GERAÇÃO DE AMOSTRA ESTRATIFICADA (10%) PARA ANÁLISE MANUAL
# ============================================================================

print("=" * 80)
print("GERAÇÃO DE AMOSTRA PARA ANÁLISE MANUAL (CODIFICAÇÃO)")
print("=" * 80)

# Calcular tamanho da amostra por vínculo (estratificada)
n_amostra = int(len(df) * PERCENTUAL_AMOSTRA)

# Amostragem estratificada por vínculo
amostra = df.groupby('Vinculo_Padronizado', group_keys=False).apply(
    lambda x: x.sample(frac=PERCENTUAL_AMOSTRA, random_state=SEED)
)

print(f"\nTamanho da amostra: {len(amostra)} registros ({len(amostra)/len(df)*100:.1f}% do total)")
print(f"\nDistribuição por vínculo:")
print(amostra['Vinculo_Padronizado'].value_counts())

# Selecionar apenas variáveis relevantes para análise
colunas_amostra = ['Vinculo_Padronizado', 'Centro_Unidade'] + list(variaveis_abertas.keys())
amostra_exportar = amostra[colunas_amostra].copy()

# Adicionar colunas para codificação manual
for var in variaveis_abertas.keys():
    amostra_exportar[f'{var}_CODIGO'] = ''  # Coluna para código temático
    amostra_exportar[f'{var}_OBS'] = ''     # Coluna para observações

# Salvar amostra
amostra_exportar.to_excel(ARQUIVO_SAIDA_AMOSTRA, index=False)
print(f"\n✅ Amostra salva em: {ARQUIVO_SAIDA_AMOSTRA}")
print(f"   Use este arquivo para codificação manual das respostas.")

---

## 10. Seleção de Citações Representativas

In [ ]:
# ============================================================================
# SELEÇÃO DE CITAÇÕES REPRESENTATIVAS POR QUESTÃO
# ============================================================================

print("=" * 80)
print("CITAÇÕES REPRESENTATIVAS POR QUESTÃO")
print("=" * 80)

todas_citacoes = []

for var, info in variaveis_abertas.items():
    print(f"\n{'='*70}")
    print(f"📌 {info['nome_curto'].upper()}")
    print(f"{'='*70}")
    
    # Selecionar citações
    citacoes = selecionar_citacoes(df[var], n_citacoes=5, min_chars=80, max_chars=400)
    
    for i, cit in enumerate(citacoes, 1):
        print(f"\n  [{i}] \"{cit}\"")
        todas_citacoes.append({
            'Questão': info['nome_curto'],
            'Variável': var,
            'Citação': cit
        })

# Criar DataFrame com citações
df_citacoes = pd.DataFrame(todas_citacoes)

In [ ]:
# ============================================================================
# CITAÇÕES ESTRATIFICADAS POR VÍNCULO (para as 3 principais questões)
# ============================================================================

print("\n" + "=" * 80)
print("CITAÇÕES POR VÍNCULO INSTITUCIONAL (QUESTÕES PRINCIPAIS)")
print("=" * 80)

questoes_principais = ['Beneficios_Percebidos', 'Riscos_Preocupacoes', 'Medidas_Uso_Etico']

citacoes_vinculo = []

for var in questoes_principais:
    info = variaveis_abertas[var]
    print(f"\n📌 {info['nome_curto']}")
    print("-" * 70)
    
    df_cit_vinculo = selecionar_citacoes_por_vinculo(df, var, n_por_vinculo=1)
    
    for _, row in df_cit_vinculo.iterrows():
        print(f"\n  [{row['Vínculo']}]")
        print(f"  \"{row['Citação'][:200]}{'...' if len(row['Citação']) > 200 else ''}\"")
        
        citacoes_vinculo.append({
            'Questão': info['nome_curto'],
            'Vínculo': row['Vínculo'],
            'Citação': row['Citação']
        })

df_citacoes_vinculo = pd.DataFrame(citacoes_vinculo)

---

## 11. Categorização Temática Preliminar

In [ ]:
# ============================================================================
# CATEGORIZAÇÃO TEMÁTICA BASEADA EM PALAVRAS-CHAVE
# ============================================================================

# Definir categorias temáticas e palavras-chave associadas
categorias_tematicas = {
    'Beneficios_Percebidos': {
        'Produtividade/Eficiência': ['produtividade', 'eficiencia', 'rapido', 'rapidez', 'tempo', 'agilidade', 'otimizar', 'automatizar'],
        'Apoio à Pesquisa': ['pesquisa', 'pesquisar', 'busca', 'buscar', 'informacao', 'dados', 'analise', 'analisar'],
        'Apoio ao Ensino/Aprendizagem': ['aprendizado', 'aprender', 'estudar', 'ensino', 'ensinar', 'didatico', 'educacao'],
        'Escrita/Redação': ['escrever', 'escrita', 'texto', 'redacao', 'revisar', 'revisao', 'corrigir', 'gramatica'],
        'Criatividade/Inovação': ['criatividade', 'criativo', 'inovacao', 'inovar', 'ideias', 'inspiracao'],
        'Acessibilidade': ['acessibilidade', 'acessivel', 'inclusao', 'inclusivo', 'democratizar']
    },
    'Riscos_Preocupacoes': {
        'Plágio/Integridade': ['plagio', 'copiar', 'copia', 'integridade', 'autoria', 'original', 'originalidade'],
        'Dependência/Preguiça': ['dependencia', 'dependente', 'preguica', 'acomodacao', 'passivo', 'pensar'],
        'Desinformação': ['desinformacao', 'fake', 'falso', 'incorreto', 'erro', 'erros', 'confiabilidade'],
        'Desemprego/Substituição': ['emprego', 'desemprego', 'substituir', 'substituicao', 'trabalho', 'profissional'],
        'Privacidade/Dados': ['privacidade', 'dados', 'seguranca', 'vazamento', 'confidencial'],
        'Viés/Discriminação': ['vies', 'discriminacao', 'preconceito', 'racismo', 'desigualdade']
    },
    'Medidas_Uso_Etico': {
        'Regulamentação/Normas': ['regulamentacao', 'regulamentar', 'norma', 'normas', 'regra', 'regras', 'politica', 'politicas'],
        'Capacitação/Formação': ['capacitacao', 'capacitar', 'formacao', 'formar', 'treinamento', 'treinar', 'curso', 'cursos'],
        'Transparência': ['transparencia', 'transparente', 'declarar', 'declaracao', 'informar', 'citar'],
        'Conscientização': ['conscientizacao', 'conscientizar', 'sensibilizar', 'orientar', 'orientacao'],
        'Fiscalização/Controle': ['fiscalizacao', 'fiscalizar', 'controle', 'monitorar', 'monitoramento', 'verificar']
    }
}


def categorizar_resposta(texto, categorias):
    """
    Categoriza uma resposta com base em palavras-chave.
    Retorna lista de categorias identificadas.
    """
    if pd.isna(texto):
        return []
    
    texto_limpo = limpar_texto(texto)
    categorias_encontradas = []
    
    for categoria, palavras in categorias.items():
        for palavra in palavras:
            if palavra in texto_limpo:
                categorias_encontradas.append(categoria)
                break  # Evitar contar mesma categoria múltiplas vezes
    
    return categorias_encontradas


# Aplicar categorização para as questões principais
print("=" * 80)
print("CATEGORIZAÇÃO TEMÁTICA PRELIMINAR (baseada em palavras-chave)")
print("=" * 80)

resultados_categorizacao = {}

for var, categorias in categorias_tematicas.items():
    info = variaveis_abertas[var]
    print(f"\n📌 {info['nome_curto']}")
    print("-" * 60)
    
    # Categorizar cada resposta
    todas_categorias = []
    for texto in df[var].dropna():
        cats = categorizar_resposta(texto, categorias)
        todas_categorias.extend(cats)
    
    # Contar frequência de cada categoria
    contagem = Counter(todas_categorias)
    n_respondentes = df[var].notna().sum()
    
    df_cat = pd.DataFrame(contagem.items(), columns=['Categoria', 'Menções'])
    df_cat = df_cat.sort_values('Menções', ascending=False)
    df_cat['% Respondentes'] = (df_cat['Menções'] / n_respondentes * 100).round(1)
    
    resultados_categorizacao[var] = df_cat
    
    print(df_cat.to_string(index=False))

In [ ]:
# ============================================================================
# GRÁFICO DE CATEGORIAS TEMÁTICAS
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 6))

cores_cat = ['#3498db', '#2ecc71', '#e74c3c']

for idx, (var, df_cat) in enumerate(resultados_categorizacao.items()):
    ax = axes[idx]
    info = variaveis_abertas[var]
    
    if len(df_cat) > 0:
        cores = plt.cm.get_cmap(['Blues', 'Greens', 'Oranges'][idx])(np.linspace(0.3, 0.8, len(df_cat)))[::-1]
        
        bars = ax.barh(df_cat['Categoria'], df_cat['% Respondentes'], color=cores, edgecolor='white')
        ax.invert_yaxis()
        ax.set_xlabel('% dos Respondentes')
        ax.set_title(info['nome_curto'], fontsize=12, fontweight='bold')
        
        for bar, pct in zip(bars, df_cat['% Respondentes']):
            ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
                    f'{pct}%', va='center', fontsize=9)

plt.suptitle('Categorias Temáticas Identificadas', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('grafico_categorias_tematicas_etapa6.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_categorias_tematicas_etapa6.png")

---

## 12. Resumo Executivo

In [ ]:
print("=" * 80)
print("RESUMO EXECUTIVO - ETAPA 6")
print("=" * 80)

print(f"""
📊 VISÃO GERAL DAS RESPOSTAS ABERTAS

   Total de questões analisadas: {len(variaveis_abertas)}
   Taxa média de resposta: {df_estatisticas['% Preenchimento'].mean():.1f}%
   Questão com maior taxa: {df_estatisticas.loc[df_estatisticas['% Preenchimento'].idxmax(), 'Nome Curto']} ({df_estatisticas['% Preenchimento'].max()}%)
   Questão com menor taxa: {df_estatisticas.loc[df_estatisticas['% Preenchimento'].idxmin(), 'Nome Curto']} ({df_estatisticas['% Preenchimento'].min()}%)

📌 PRINCIPAIS TEMAS IDENTIFICADOS
""")

for var, df_cat in resultados_categorizacao.items():
    info = variaveis_abertas[var]
    if len(df_cat) > 0:
        top_cat = df_cat.iloc[0]
        print(f"   • {info['nome_curto']}: \"{top_cat['Categoria']}\" ({top_cat['% Respondentes']}%)")

print(f"""
📌 TERMOS MAIS FREQUENTES (GLOBAL)
""")

for var, info in list(variaveis_abertas.items())[:3]:
    if var in frequencias_termos and len(frequencias_termos[var]) > 0:
        top_termos = frequencias_termos[var].head(3)['Termo'].tolist()
        print(f"   • {info['nome_curto']}: {', '.join(top_termos)}")

print(f"""
📌 MATERIAIS GERADOS PARA ANÁLISE MANUAL

   • Amostra de {len(amostra)} respostas ({PERCENTUAL_AMOSTRA*100:.0f}%) para codificação
   • {len(df_citacoes)} citações representativas selecionadas
   • {len(df_citacoes_vinculo)} citações estratificadas por vínculo
""")

---

## 13. Salvar Resultados

In [ ]:
# Salvar todas as tabelas em um arquivo Excel com múltiplas abas
with pd.ExcelWriter(ARQUIVO_SAIDA_EXCEL, engine='openpyxl') as writer:
    # Estatísticas gerais
    df_estatisticas.to_excel(writer, sheet_name='Estatisticas_Gerais', index=False)
    
    # Frequência de termos
    for var, freq in frequencias_termos.items():
        nome_aba = f'Termos_{variaveis_abertas[var]["nome_curto"][:15]}'
        freq.to_excel(writer, sheet_name=nome_aba, index=False)
    
    # Frequência de bigramas
    for var, freq in frequencias_bigramas.items():
        if len(freq) > 0:
            nome_aba = f'Bigr_{variaveis_abertas[var]["nome_curto"][:15]}'
            freq.to_excel(writer, sheet_name=nome_aba, index=False)
    
    # Categorização temática
    for var, df_cat in resultados_categorizacao.items():
        nome_aba = f'Cat_{variaveis_abertas[var]["nome_curto"][:15]}'
        df_cat.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"✅ Análises salvas em: {ARQUIVO_SAIDA_EXCEL}")

In [ ]:
# Salvar citações representativas
with pd.ExcelWriter(ARQUIVO_SAIDA_CITACOES, engine='openpyxl') as writer:
    df_citacoes.to_excel(writer, sheet_name='Citacoes_Gerais', index=False)
    df_citacoes_vinculo.to_excel(writer, sheet_name='Citacoes_por_Vinculo', index=False)

print(f"✅ Citações salvas em: {ARQUIVO_SAIDA_CITACOES}")

In [ ]:
# Gerar relatório em Markdown
relatorio = f"""# RELATÓRIO DE ANÁLISE QUALITATIVA DAS RESPOSTAS ABERTAS
## Etapa 6 — Consulta à Comunidade UFPE sobre o Uso de IA

**Data de execução:** {datetime.now().strftime("%d/%m/%Y às %H:%M")}

---

## 1. VISÃO GERAL DAS RESPOSTAS

| Questão | Respostas | % Preenchimento | Comp. Médio |
|---------|-----------|-----------------|-------------|
"""

for _, row in df_estatisticas.iterrows():
    relatorio += f"| {row['Nome Curto']} | {row['Respostas']} | {row['% Preenchimento']}% | {row['Comp. Médio (chars)']:.0f} chars |\n"

relatorio += f"""
---

## 2. CATEGORIAS TEMÁTICAS IDENTIFICADAS

### 2.1 Benefícios Percebidos

| Categoria | Menções | % Respondentes |
|-----------|---------|----------------|
"""

if 'Beneficios_Percebidos' in resultados_categorizacao:
    for _, row in resultados_categorizacao['Beneficios_Percebidos'].iterrows():
        relatorio += f"| {row['Categoria']} | {row['Menções']} | {row['% Respondentes']}% |\n"

relatorio += f"""
### 2.2 Riscos e Preocupações

| Categoria | Menções | % Respondentes |
|-----------|---------|----------------|
"""

if 'Riscos_Preocupacoes' in resultados_categorizacao:
    for _, row in resultados_categorizacao['Riscos_Preocupacoes'].iterrows():
        relatorio += f"| {row['Categoria']} | {row['Menções']} | {row['% Respondentes']}% |\n"

relatorio += f"""
### 2.3 Medidas para Uso Ético

| Categoria | Menções | % Respondentes |
|-----------|---------|----------------|
"""

if 'Medidas_Uso_Etico' in resultados_categorizacao:
    for _, row in resultados_categorizacao['Medidas_Uso_Etico'].iterrows():
        relatorio += f"| {row['Categoria']} | {row['Menções']} | {row['% Respondentes']}% |\n"

relatorio += f"""
---

## 3. CITAÇÕES REPRESENTATIVAS

"""

for var, info in list(variaveis_abertas.items())[:3]:
    relatorio += f"### {info['nome_curto']}\n\n"
    citacoes_var = df_citacoes[df_citacoes['Variável'] == var]['Citação'].head(3).tolist()
    for i, cit in enumerate(citacoes_var, 1):
        cit_truncada = cit[:300] + '...' if len(cit) > 300 else cit
        relatorio += f"> *\"{cit_truncada}\"*\n\n"

relatorio += f"""
---

## 4. TERMOS MAIS FREQUENTES

"""

for var, info in list(variaveis_abertas.items())[:3]:
    relatorio += f"### {info['nome_curto']}\n\n"
    if var in frequencias_termos and len(frequencias_termos[var]) > 0:
        relatorio += "| # | Termo | Frequência |\n|---|-------|------------|\n"
        for _, row in frequencias_termos[var].head(10).iterrows():
            relatorio += f"| {row['Ranking']} | {row['Termo']} | {row['Frequência']} |\n"
    relatorio += "\n"

relatorio += f"""
---

## 5. MATERIAIS PARA ANÁLISE MANUAL

| Arquivo | Descrição |
|---------|----------|
| `{ARQUIVO_SAIDA_AMOSTRA}` | Amostra de {len(amostra)} respostas ({PERCENTUAL_AMOSTRA*100:.0f}%) para codificação manual |
| `{ARQUIVO_SAIDA_CITACOES}` | Citações representativas selecionadas |

### Instruções para Codificação Manual

1. Abra o arquivo `{ARQUIVO_SAIDA_AMOSTRA}`
2. Para cada resposta, preencha a coluna `_CODIGO` com a categoria temática identificada
3. Use a coluna `_OBS` para observações adicionais
4. Recomenda-se que dois codificadores independentes realizem a análise
5. Calcule o índice Kappa para verificar a concordância entre codificadores

---

## 6. ARQUIVOS GERADOS

| Arquivo | Descrição |
|---------|----------|
| `{ARQUIVO_SAIDA_EXCEL}` | Análise de frequência de termos e categorias |
| `{ARQUIVO_SAIDA_AMOSTRA}` | Amostra para análise manual |
| `{ARQUIVO_SAIDA_CITACOES}` | Citações representativas |
| `grafico_comprimento_respostas_etapa6.png` | Distribuição do comprimento |
| `grafico_frequencia_termos_etapa6.png` | Termos mais frequentes |
| `grafico_nuvens_completo_etapa6.png` | Painel de nuvens de palavras |
| `grafico_nuvem_[variavel]_etapa6.png` | Nuvens individuais |
| `grafico_categorias_tematicas_etapa6.png` | Categorias temáticas |

---

## 7. LIMITAÇÕES E PRÓXIMOS PASSOS

**Limitações desta análise automática:**
- A categorização por palavras-chave é aproximada e deve ser validada manualmente
- Nuances e contexto podem ser perdidos na análise automatizada
- Recomenda-se análise manual da amostra para refinamento das categorias

**Próximos passos recomendados:**
1. Realizar codificação manual da amostra de 10%
2. Refinar o codebook com base nos achados
3. Calcular índice Kappa entre codificadores
4. Expandir análise para toda a base após validação

---

*Relatório gerado automaticamente em {datetime.now().strftime("%d/%m/%Y às %H:%M")}*
"""

# Salvar relatório
with open(ARQUIVO_SAIDA_RELATORIO, "w", encoding='utf-8') as f:
    f.write(relatorio)

print(f"✅ Relatório salvo em: {ARQUIVO_SAIDA_RELATORIO}")

In [ ]:
print("\n" + "=" * 70)
print("ETAPA 6 CONCLUÍDA COM SUCESSO!")
print("=" * 70)
print(f"""
📁 ARQUIVOS GERADOS:

   📊 Dados:
      • {ARQUIVO_SAIDA_EXCEL}
      • {ARQUIVO_SAIDA_AMOSTRA}
      • {ARQUIVO_SAIDA_CITACOES}

   📈 Gráficos:
      • grafico_comprimento_respostas_etapa6.png
      • grafico_frequencia_termos_etapa6.png
      • grafico_nuvens_completo_etapa6.png
      • grafico_nuvem_[variavel]_etapa6.png (7 arquivos)
      • grafico_categorias_tematicas_etapa6.png

   📝 Relatório:
      • {ARQUIVO_SAIDA_RELATORIO}

▶️  PRÓXIMA ETAPA: Etapa 7 - Síntese e Elaboração de Recomendações

⚠️  IMPORTANTE: Antes de prosseguir para a Etapa 7, recomenda-se:
      1. Revisar a amostra gerada ({ARQUIVO_SAIDA_AMOSTRA})
      2. Realizar codificação manual com dois codificadores
      3. Calcular índice Kappa para validação
""")

---

## Resumo dos Arquivos

### Entrada
| Arquivo | Origem |
|---------|--------|
| `dados_limpos_etapa1.xlsx` | Etapa 1 |

### Saída
| Arquivo | Descrição |
|---------|----------|
| `qualitativa_etapa6.xlsx` | Análise de frequência de termos e categorias temáticas |
| `amostra_analise_etapa6.xlsx` | Amostra de 10% para codificação manual |
| `citacoes_representativas_etapa6.xlsx` | Citações selecionadas por questão e vínculo |
| `grafico_comprimento_respostas_etapa6.png` | Distribuição do comprimento das respostas |
| `grafico_frequencia_termos_etapa6.png` | Top 10 termos por questão |
| `grafico_nuvens_completo_etapa6.png` | Painel com todas as nuvens de palavras |
| `grafico_nuvem_[variavel]_etapa6.png` | Nuvens individuais (7 arquivos) |
| `grafico_categorias_tematicas_etapa6.png` | Categorias temáticas identificadas |
| `relatorio_qualitativa_etapa6.md` | Relatório completo da análise qualitativa |

---

**Próxima etapa:** Etapa 7 - Síntese e Elaboração de Recomendações